# TensorFlow Tutorial

In [1]:
import numpy as np
import tensorflow as tf


def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

## 1. Get available GPU

In [ ]:
''' Check if gpu is available '''
# (1):
print("Method (1) stdout: ")
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# (2)
print("Method (2) stdout: ")
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))
    
# (3)
print("\nMethod (3) stdout: ")
with tf.Session() as sess:
    devices = sess.list_devices() 
    print(devices)
    
# (4)
print("\nMethod (4) stdout: ")
from tensorflow.python.client import device_lib
from pprint import pprint
pprint(device_lib.list_local_devices())

## 2. Conv2d

In [5]:
# Use the same `input` and `weights` to test implementation.
conv2d_input_np = np.random.normal(size=[1, 3, 3, 5])
conv2d_filters_np = np.random.normal(size=[1, 1, 5, 1])

conv2d_input = tf.constant(conv2d_input_np)
conv2d_filters = tf.constant(conv2d_filters_np)

# (1)
# tf.nn.conv2d
def tf_nn_conv2d(input, filters):
    # Use function to avoid naming conflicts.
    # tf.Variable (class): 
    #     A variable maintains state in the graph across calls to `run()`. 
    #     When you launch the graph, variables have to be explicitly 
    #     initialized before you can run Ops that uses their value.
    input = tf.Variable(input)
    filters = tf.Variable(filters)
    # No `bias` added for `tf.nn.conv2d`.
    nn_conv2d_op = tf.nn.conv2d(
        input=input,
        filters=filters,
        strides=[1,1,1,1],
        padding='VALID',
        data_format='NHWC',
        dilations=None,
        name=None
    )
    
    # Add an Op to initialize global variables.
    init_op = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init_op)
        res = sess.run(nn_conv2d_op)
    return res

nn_conv2d_res = tf_nn_conv2d(conv2d_input, conv2d_filters)

# (2)
# tf.layers.conv2d
def tf_layers_conv2d(input, filters):
    input = tf.Variable(input)
    
    kernel_initializer = tf.constant_initializer(value=tf.Session().run(filters))
    
    # tf.layers.conv2d:
    #     Functional interface for the 2D convolution layer.
    # Arguments:
    #     inputs: 
    #         Tensor input.
    #     filters: 
    #         Integer, the dimensionality of the output space (i.e. The number 
    #         of filters in the convolution).
    #     kernel_size: 
    #         An integer or tuple/list of 2 integers, specifying the height and 
    #         width of the 2D convolution window. Can be a single integer to specify
    #         the same value for all spatial dimensions.
    layers_conv2d_op = tf.layers.conv2d(
        inputs=input, 
        filters=1, 
        kernel_size=[1,1], 
        strides=1,
        padding='VALID', 
        use_bias=False,
        kernel_initializer=kernel_initializer,
        data_format="channels_last"
    )
    
    # Add an Op to initialize global variables.
    init_op = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        with tf.device("/gpu:0"):
            sess.run(init_op)
            res = sess.run(layers_conv2d_op) 
    return res

layers_conv2d_res = tf_layers_conv2d(conv2d_input, conv2d_filters)

print(
    "conv2d between `tf.nn.conv2d` and `tf.layers.conv2d` is: ", \
    rel_error(nn_conv2d_res, layers_conv2d_res)
)

conv2d between `tf.nn.conv2d` and `tf.layers.conv2d` is:  0.0


In [4]:
# (3)
# tf.keras.layers
tf.keras.backend.clear_session()  # For easy reset of notebook state.

# Use the same `input` and `weights` to test implementation.
# To avoid `Tensor is not an element in the graph` error.
conv2d_input = tf.constant(conv2d_input_np)
conv2d_filters = tf.constant(conv2d_filters_np)


def tf_keras_conv2d(input, filters):
    # tf.keras.Input:
    #     Used to instantiate a Keras tensor.
    # Arguments:
    #     shape:
    #         A shape tuple (integers), not including the batch size. For instance,
    #         `shape=(32,)` indicates that the expected input will be batches of 
    #         32-dimensional vectors. Elements of this tuple can be None; `None`
    #         elements represent dimensions where the shape is not known.
    #     tensor:
    #         Optional existing tensor to wrap into the Input layer. If set, the layer
    #         will not create a placeholder tensor.
    input = tf.keras.Input(
        shape=input.get_shape().as_list(),
        tensor=input
    )
    
    kernel_initializer = tf.constant_initializer(value=tf.Session().run(filters))
    
    # tf.keras.layers.Conv2D:
    #     This layer creates a convolution kernel that is convolved with the layer 
    #     input to produce a `tensor` of outputs.
    # Arguments:
    #     filters: 
    #         Integer, the dimensionality of the output space (i.e. The number 
    #         of filters in the convolution).
    #     kernel_size: 
    #         An integer or tuple/list of 2 integers, specifying the height and 
    #         width of the 2D convolution window. Can be a single integer to specify
    #         the same value for all spatial dimensions.
    keras_conv2d = tf.keras.layers.Conv2D(
        filters=1, 
        kernel_size=[1,1], 
        strides=1,
        padding='VALID', 
        use_bias=False,
        kernel_initializer=kernel_initializer,
        data_format="channels_last"
    )(input)
    
    model = tf.keras.Model(inputs=input, outputs=keras_conv2d)
    # model.summary()
    
    # Add an Op to initialize global variables.
    init_op = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init_op)
        res = sess.run(model(input, training=False))
    return res

keras_conv2d_res = tf_keras_conv2d(conv2d_input, conv2d_filters)

print(
    "conv2d between tf.nn.conv2d and tf.keras.layers.Conv2d is: ", \
    rel_error(nn_conv2d_res, keras_conv2d_res)
)
print(
    "conv2d between tf.layers.conv2d and tf.keras.layers.Conv2d is: ", \
    rel_error(layers_conv2d_res, keras_conv2d_res)
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
conv2d between tf.nn.conv2d and tf.keras.layers.Conv2d is:  0.0
conv2d between tf.layers.conv2d and tf.keras.layers.Conv2d is:  0.0


In [5]:
# (4)
# tf.slim.conv2d
tf.contrib.slim

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



<module 'tensorflow_core.contrib.slim' from '/home/wangchu/anaconda3/envs/machine_learning/lib/python3.6/site-packages/tensorflow_core/contrib/slim/__init__.py'>

## 3. Batch Normalization
![](batch_norm_cs231n.png)

### 3.1 Without Moving Average

In [2]:
from tensorflow.python.framework import constant_op
from tensorflow.python.ops import math_ops, nn_impl

In [3]:
x_shape = [3, 5, 4, 2]    # NHWC
param_shape = [2]    # C
x_val = np.random.random_sample(x_shape).astype(np.float32)
m_val = np.random.random_sample(param_shape).astype(np.float32)
v_val = np.random.random_sample(param_shape).astype(np.float32)
beta_val = np.random.random_sample(param_shape).astype(np.float32)
gamma_val = np.random.random_sample(param_shape).astype(np.float32)
epsilon = 1e-8

# (1)
# low level ops

# https://github.com/tensorflow/tensorflow/blob/50396e0d8e7c6bf518d4eff28766c872ebd6f3d9/tensorflow/python/ops/nn_batchnorm_test.py
def _npBatchNorm(x, m, v, beta, gamma, epsilon,
                scale_after_normalization, shift_after_normalization):
    y = (x - m) / np.sqrt(v + epsilon)
    if scale_after_normalization:
        y = y * gamma
    if shift_after_normalization:
        y = y + beta
    return y

def _opsBatchNorm(x, m, v, beta, gamma, epsilon,
                 scale_after_normalization, shift_after_normalization):
    y = (x - m) * math_ops.rsqrt(v + epsilon)
    if scale_after_normalization:
        y = gamma * y
    if shift_after_normalization:
        y = y + beta
    return y

def _tfBatchNormV2(x, m, v, beta, gamma, epsilon,
                  scale_after_normalization, shift_after_normalization):
    """New implementation."""
    return nn_impl.batch_normalization(x, m, v, beta if
                                       shift_after_normalization else None,
                                       gamma if scale_after_normalization else
                                       None, epsilon)

def testBatchNorm(x_val, m_val, v_val, beta_val, gamma_val, epsilon):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    with tf.Session(config=config) as sess:
        x = constant_op.constant(x_val, name='x')
        m = constant_op.constant(m_val, name='m')
        v = constant_op.constant(v_val, name='v')
        beta = constant_op.constant(beta_val, name='beta')
        gamma = constant_op.constant(gamma_val, name='gamma')
        
        scale_after_normalization = True
        shift_after_normalization = True
        
        bn2 = _tfBatchNormV2(x, m, v, beta, gamma, epsilon,
                             scale_after_normalization,
                             shift_after_normalization)
        
        on = _opsBatchNorm(x, m, v, beta, gamma, epsilon,
                          scale_after_normalization,
                          shift_after_normalization)
        
        np_bn = _npBatchNorm(x_val, m_val, v_val, beta_val, gamma_val,
                             epsilon, scale_after_normalization,
                             shift_after_normalization)
        
        tf_bn_v2, ops_bn = sess.run([bn2, on])
        
        print(rel_error(np_bn, ops_bn))
        print(rel_error(np_bn, tf_bn_v2))
        print(rel_error(tf_bn_v2, ops_bn))
        return np_bn, ops_bn, tf_bn_v2

np_bn, ops_bn, tf_bn_v2 = testBatchNorm(x_val, m_val, v_val, beta_val, gamma_val, epsilon)

4.8294014e-06
1.4568574e-06
4.8294014e-06


In [4]:

# (2)
# tf.nn.batch_normalization

# https://stackoverflow.com/questions/33949786/how-could-i-use-batch-normalization-in-tensorflow
def tf_nn_batch_norm(x_val, m_val, v_val, beta_val, gamma_val, epsilon):
    x = constant_op.constant(x_val, name='x')
    
    mean = tf.Variable(tf.constant(m_val), trainable=False)
    variance = tf.Variable(tf.constant(v_val), trainable=False)
    
    beta = tf.Variable(tf.constant(beta_val))
    gamma = tf.Variable(tf.constant(gamma_val))
           
    # Add an Op to initialize global variables.
    init_op = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init_op)
        tf_nn_batch_norm_op = tf.nn.batch_norm_with_global_normalization(
            x, mean, variance, beta, gamma, epsilon, True)
        res = sess.run(tf_nn_batch_norm_op)
    return res
    
tf_nn_bn = tf_nn_batch_norm(x_val, m_val, v_val, beta_val, gamma_val, epsilon)

# Note: no moving average is applied.
print(rel_error(tf_nn_bn, np_bn))
print(rel_error(tf_nn_bn, tf_bn_v2))
print(rel_error(tf_nn_bn, ops_bn))

1.4568574e-06
0.0
4.8294014e-06


### 3.2 With Moving Average

In [30]:
# tf.train.ExponentialMovingAverage (class):
#     Maintains moving averages of variables by employing an exponential decay.
#     The `apply()` method adds shadow copies of trained variables and add ops that 
#     maintain a moving average of the trained variables in their shadow copies. It
#     is used when building the training model. The ops that maintain moving averages
#     are typically run after each training step. The `average()` and `average_name()`
#     methods give access to the shadow variables and their names.
# Args:
#     decay: 
#         Float. The decay to use.
# Methods:
#     apply(var_list=None):
#         Maintains moving averages of variables.
#         Args:
#             var_list:
#                 A list of Variable or Tensor objects. The variables and Tensors must
#                 be of type float16, float32, or float64.
#         Returns:
#             An Operation that updates the moving averages.
#     average(var):
#         Returns the `Variable` holding the average of `var`.
#         Args:
#             var:
#                 A `Variable` object.   
#         Returns:
#             A `Variable` object or `None` if the moving average of `var` is not maintained.
ewma_trainer = tf.train.ExponentialMovingAverage(decay=0.99)


# tf.nn.moments:
#     Calculates the mean and variance of `x`. The mean and variance are calculated
#     by aggregating the contents of `x` across `axes`. If `x` is 1-D and `axes=[0]`
#     this is just the mean and variance of a vector.
#     When using these moments for batch normalization (tf.nn.batch_normalization):
#         - for so-called "global normalization", used with convolutional filters
#           with shape `[batch, height, width, depth]`, pass `axes=[0, 1, 2]`.
#         - for simple batch normalization pass `axes=[0]` (batch only).
# Args:
#     `x`: A `Tensor`
#     `axes`: Array of ints. Axes along which to compute mean and variance.
# Returns:
#     Two `Tensor` objects: `mean` and `variance`.
ma_mean, ma_variance = tf.nn.moments(x, [0, 1, 2])

ewma_trainer.apply([self.mean, self.variance])


# tf.Variable (class)
# Methods:
#     assign(value):
#     Assigns a new value to the variable. This is essentially a shortcut for `tf.assign`.


# tf.assign(ref, value):
#     This operation outputs a Tensor that holds the new value of `ref` after the value
#     has been assigned. This makes it easier to chain operations that need to use the 
#     reset value.
# Args:
#     `ref`: 
#         A mutable `Tensor`. Should be from a `Variable` node. May be uninitialized.
#     `value`:
#         A `Tensor`. Must have the same shape and dtypes as `ref`. The value to be
#         assigned to the variable.
# Returns:
#     A `Tensor` that will hold the new value of `ref` after the assignment has completed.
assign_mean = mean.assign(ma_mean)
assign_variance = variance.assign(ma_variance)

NameError: name 'x' is not defined